In [1]:
import time
import tkinter as tk
from tkinter import ttk		# For Notebook
from tkinter import *			# For TopLevel() and Text

from gethistory import GetHistory
from globals import nomRate, sausageRate, noteBookWidth, noteBookHeight, settingsShared

import graphTab as gt

try:
	from icecream import ic
except ImportError:  # Graceful fallback if IceCream isn't installed.
	ic = lambda *a: None if not a else (a[0] if len(a) == 1 else a)  # noqa
ic.enable()
# Other things similar to Icecream:
#   (See https://medium.com/swlh/all-python-debugging-tools-you-need-to-know-in-2020-e3ff66b8f318)
#   objprint, PySnooper, hunter, Watchpoints, VizTracer (and more)

# results = Results(0,'',0,0,0,0,0,0,0,'',0,0,'','')
# results.dataLength = 18
# results.timeNow		 = "11:32"
# results.tempNow		 = 102.3
# results.last1			 = 12
# results.last2			 = 13
# results.last4			 = 14
# results.last8			 = 15
# results.last16		 = 16
# results.want			 = 145
# results.atTime		 = "11:45"
# results.need			 = 26
# results.lateness	 = 27
# results.finishTimeThis	= "11:00"
# results.finishTimeTarget= "12:13"


cruiseTemp = 170
day1Temp = 145
day1Finish = "14:30"
cruiseTime = "11:45"		# Might be overridden

# root window
_widthCnv = noteBookWidth+15 
_heightCnv = noteBookHeight+110
geomStr = f'{_widthCnv}x{_heightCnv}'
root = tk.Tk()
root.geometry(geomStr)
root.title('Waratah Warm-up')

# create a notebook

def rememberConfig(virtualEvent):
	continuedBelow(virtualEvent)		# . . . for programmer's convenience (while keeping interpretter happy)
	 

notebook = ttk.Notebook(root, height=noteBookHeight, width=noteBookWidth)
notebook.pack(pady=10, expand=True)
notebook.bind("<<NotebookTabChanged>>", rememberConfig)		# Just calls continuedBelow() in the "Analyse" code

# create frames
frSetup   = ttk.Frame(notebook, width=300, height=500)		# Dimensions overruled by "notebook = ttk.Notebook( . . . " above
frAnalyse = ttk.Frame(notebook, width=300, height=500)
frGraph   = ttk.Frame(notebook, width=300, height=500)

frSetup.pack  (fill='both', expand=True)
frAnalyse.pack(fill='both', expand=True)
frGraph.pack  (fill='both', expand=True)

# add frames to notebook

notebook.add(frSetup,   text='Setup')
notebook.add(frAnalyse, text='Analyse')
notebook.add(frGraph,   text='Graph')
notebook.pack(expand = 1, fill ="both")

canvas = Canvas(frGraph)
canvas.create_text(100, 100, anchor=W, angle=90, text="130 deg C")
canvas.create_text(10, 10, anchor=W, angle=90, text="At 10,10")


canvas.pack(fill = BOTH, expand = 1)
canvas.update()


########################################
# "Setup" tab
########################################

# radio buttons
realOrHistRadioPick = tk.IntVar()

# Declare some globals (with junk vlues)
finishTemp = -2
isRealTimeSelected =True


# Called when a radio button is clicked
# Updates the  text boxes on the Setup page with the selected configuration
def radioCall():
	global isRealTimeSelected, finishTemp

	#ic(realOrHistRadioPick.get())
	index = int(realOrHistRadioPick.get())
	isRealTimeSelected = radioDefinition[index][1] == "Real time"
	isFirstDay.set(radioDefinition[index][4])
	historyDate.set(radioDefinition[index][2])
	historyTime.set(radioDefinition[index][3])
	comment.set(radioDefinition[index][5])

	if isFirstDay.get(): 
		finishTemp.set(day1Temp)
		finishTime.set(day1Finish)
	else:
		finishTemp.set(cruiseTemp)
		finishTime.set(cruiseTime)

	setFinish()


def setFinish():
	global cruiseTemp, day1Temp, day1Finish, cruiseTime, finishTemp, isRealTimeSelected

	temperatureInfo.set(f"Warm-up finish at {finishTime.get()}, with temperature {finishTemp.get()}")
# End of SetFinish()


row = 0
ttk.Label(frSetup, text ="Setup page", font=("Arial", 18)).grid(column = 0, row = row, columnspan=2)
row +=1


radioDefinition = [
	(0, "Real time",     "Today",      "Now",   True,  ""),
	(1, "History day 1", "2021-06-18", "10:21", True,  "Too slow at first, then too fast"), 
	(2, "History day 2", "2021-06-19", "10:21", False, "Late from about 8am till 10:55, then early")
]

i = 0
for option in (radioDefinition):
	r = ttk.Radiobutton(
			frSetup,
			text=option[1],
			value=option[0],
			variable=realOrHistRadioPick,
			command=radioCall
	).grid(column = 0, row = row, sticky=W, padx=9, columnspan=2)
	#r.pack(fill='x', padx=5, pady=5)
	row +=1



isFirstDay = BooleanVar()
Checkbutton(frSetup, text='First day', variable=isFirstDay, command=setFinish).grid(row=row, column=1, sticky=W, padx=9)
row +=1

historyDate = StringVar()
Label(frSetup, text='Date:').grid(row=row, column=0, sticky=E, padx=9)
Entry(frSetup, textvariable = historyDate).grid(row=row, column=1)
row +=1

historyTime = StringVar()
Label(frSetup, text='Time:').grid(row=row, column=0, sticky=E, padx=9)
Entry(frSetup, textvariable = historyTime).grid(row=row, column=1)
row +=1

finishTime = StringVar()
Label(frSetup, text='Finish time:').grid(row=row, column=0, sticky=E, padx=9)
Entry(frSetup, textvariable=finishTime).grid(row=row, column=1)
row +=1

finishTemp = IntVar()
Label(frSetup, text='Finish temp:').grid(row=row, column=0, sticky=E, padx=9)
Entry(frSetup, textvariable=finishTemp).grid(row=row, column=1)
row +=1

comment = StringVar()
Label(frSetup, text='Comment:').grid(row=row, column=0, sticky=E, padx=9)
Entry(frSetup, textvariable = comment).grid(row=row, column=1)
row +=1

Label(frSetup, text="  .  .  .  .  .  .  .").grid(row=row, column=1, sticky=W, columnspan=2, padx=9)
row +=1

temperatureInfo = StringVar()
Label(frSetup, textvariable = temperatureInfo).grid(row=row, column=0, sticky=W, columnspan=2, padx=9)
row +=1

rateTxt = f"Target rate = {nomRate}, Sausage rate = {round(sausageRate)}" 
Label(frSetup, text=rateTxt).grid(row=row, column=0, sticky=W, columnspan=2, padx=9)
row +=1

# Make RealTime the default radio-button option
realOrHistRadioPick.set(0)
#ic(f"Setting initial radio index to {realOrHistRadioPick.get()}")
radioCall()		# Update fields with values corresponding to the initial radio-button selection



########################################
# "Analyse" tab
########################################
lastTab = 0
setupHasChanged = True

def continuedBelow(virtualEvent):		# Called by rememberConfig() when tab changes
	global nomRate, settingsShared, finishTemp, isRealTimeSelected, setupHasChanged, lastTab

	actionNeeded = lastTab == 0
	lastTab = virtualEvent.widget.index("current")

	if actionNeeded: 	# We have just left the set-up tab (0), so save the new settings
		#ic(f"setupHasChanged was {setupHasChanged}, now True")
		setupHasChanged = True
		settings = settingsShared		# mmm ... Let's change the name
		settings.isFirstDay = isFirstDay.get()
		settings.plan_riseRate = nomRate
		settings.isRealTime = isRealTimeSelected

		dateAndTimeStr = historyDate.get() + ' ' + historyTime.get()
		if dateAndTimeStr == "Today Now":
			dateAndTimeStr = time.strftime("%Y-%m-%d %H:%M", time.localtime(time.time()))
		settings.dateAndTimeStr = dateAndTimeStr

		setFinish()
		settings.finishTemp = finishTemp.get()		# To ensure that finishTemp is set

		settings.finishTime = finishTime.get()
		#ic(settings.finishTime)

		strDayNow = "Today"
		if settings.isRealTime:
			tempTxt ="Real time"
		else:
			tempTxt ="Historical"
			strDayNow = historyDate.get()

		realOrHist.set(tempTxt + " analysis")

		if settings.isFirstDay:
			strDayNow = strDayNow + ", first day."
		else:
			strDayNow = strDayNow + ", second day."

		strMission = f"End temperature = {settings.finishTemp} at {settings.finishTime}" 
		endConditions.set(strDayNow + ' ' + strMission)
		settings.strDayNow = strDayNow
		settings.strMission = strMission

		## Decided not to do this any more
		# if settings.isFirstDay:
		# 	lateness.set("n/a")
		# else:
		# 	endAtCurrent.set("n/a")
		# 	endAtTarget.set("n/a")

		##ic(settings)

		####################################
		##ic(results)

		getAndDisplayResults()


def dec1(self, floatNum):
	return format(floatNum, '0.1f')


def getAndDisplayResults():		# This is a GUI 'command' so it can't have parameters
	global finishTemp, setupHasChanged, settingsShared, canvas, _widthCnv, _heightCnv
	#ic("Refreshing")

	history = GetHistory(setupHasChanged, settingsShared)

	#ic(history.finishTemp)
	results = history.results
	#ic(results)
	if results.failureReason != '':
		tempNow.set(f"ERROR: {results.failureReason}")
	elif results.dataLength == 0:
		tempNow.set("Internet fail?")
	else:
		setupHasChanged = False

		tempNow.set(results.tempNow)
		timeNow.set(results.timeNow)
		historyTime.set(results.timeNow)		# Upate the time on the setting page
		#ic(results.timeNow)

		last4.set(f"{results.last4}  ({results.last2}, {results.last4}, {results.last8})")
		if float(results.last4) > sausageRate:
			actualLabel.config(fg ='red')
		elif float(results.last4) > nomRate:
			actualLabel.config(fg ='DarkOrange1')
		else:
			actualLabel.config(fg ='green')

		need.set(results.need)
		#ic(results.need)

		#if settingsShared.isFirstDay:
		endAtCurrent.set(results.finishTimeThis)
		endAtTarget.set(results.finishTimeTarget)
		#else:
		lateness.set(results.lateness)

		#
		tempGraph = gt.graphTab()
		tempGraph.initUI(canvas, _widthCnv, _heightCnv, results)



row = 0
realOrHist = StringVar()
ttk.Label(frAnalyse, textvariable=realOrHist, font=("Arial", 18)).grid(column = 0, row = row, columnspan=3, pady=7)
row += 1

endConditions = StringVar()
Label(frAnalyse, textvariable=endConditions).grid(row=row, sticky=W, columnspan=3, padx=9)
row +=1

tempTxt = f"Target rate = {nomRate}, Sausage rate = {round(sausageRate)}" 
Label(frAnalyse, text=tempTxt).grid(row=row, sticky=W, columnspan=3, padx=9)
row +=1

Label(frAnalyse, text="    - - - - - - - - - - - - - - - - - - - - - - - -").grid(row=row, sticky=W, columnspan=3, padx=9)
row +=1

timeNow = StringVar()
Label(frAnalyse, text="Time:", font=("Arial", 10)).grid(row=row, column=0, sticky=E, padx=9)
Label(frAnalyse, textvariable=timeNow, font=("Arial", 10)).grid(row=row, column=1, sticky=W)
row +=1

tempNow = StringVar()
Label(frAnalyse, text="Temp:", font=("Arial", 10)).grid(row=row, column=0, sticky=E, padx=9)
Label(frAnalyse, textvariable=tempNow, font=("Arial", 10)).grid(row=row, column=1, sticky=W)
row +=1

last4 = StringVar()
Label(frAnalyse, text="Deg/hr (actual):", font=("Arial", 10)).grid(row=row, column=0, sticky=E, padx=9)
actualLabel = Label(frAnalyse, textvariable=last4, font=("Arial", 10))
actualLabel.grid(row=row, column=1, sticky=W)
row +=1

need = StringVar()
Label(frAnalyse, text="Deg/hr to meet target:", font=("Arial", 10)).grid(row=row, column=0, sticky=E, padx=9)
Label(frAnalyse, textvariable=need, font=("Arial", 10)).grid(row=row, column=1, sticky=W)
row +=1

lateness = StringVar()
Label(frAnalyse, text="Lateness (minutes):", font=("Arial", 10)).grid(row=row, column=0, sticky=E, padx=9)
Label(frAnalyse, textvariable=lateness, font=("Arial", 10)).grid(row=row, column=1, sticky=W)
row +=1

endAtCurrent = StringVar()
Label(frAnalyse, text="End time @ current rate:", font=("Arial", 10)).grid(row=row, column=0, sticky=E, padx=9)
Label(frAnalyse, textvariable=endAtCurrent, font=("Arial", 10)).grid(row=row, column=1, sticky=W)
row +=1

endAtTarget = StringVar()
Label(frAnalyse, text="End time @ target rate:", font=("Arial", 10)).grid(row=row, column=0, sticky=E, padx=9)
Label(frAnalyse, textvariable=endAtTarget, font=("Arial", 10)).grid(row=row, column=1, sticky=W)
row +=1

Label(frAnalyse, text="    - - - - - - - - - - - - - - - - - - - ").grid(row=row, sticky=W, columnspan=3, padx=9)
row +=1

Button(frAnalyse, text="  Refresh  ", font=("Arial", 9), command=getAndDisplayResults).grid(row=row, column=1, sticky=W)
row +=1



########################################
# "Graph" tab
########################################
row = 0
buttonGraph = Button(frGraph, text="", width=2, height=4, bg="#ddd",
												command=getAndDisplayResults)
buttonGraph.place(x=170, y=35)
row +=1


root.mainloop()		# Run the GUI

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\azzle\documents\bigsoftware\python38-32\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\azzle\AppData\Local\Temp/ipykernel_28632/2883467170.py", line 53, in rememberConfig
    continuedBelow(virtualEvent)		# . . . for programmer's convenience (while keeping interpretter happy)
NameError: name 'continuedBelow' is not defined


Reading top . . .


ic| logItemParam: LogItem(dateStr='2021-08-30', timeStampEnd=1630319850448.939, feedId=424220, length=18, fatalError=False, errorStr='', points=[], latestPoint=PointTT(time=0, temp=0))
ic| temperatureLog.py:223 in fillLogItem() at 20:37:30.790


Url = http://emoncms.org/feed/data.json?id=424220&start=1630318770448.939&end=1630319850448.939&interval=5&skipmissing=1&limitinterval=1&apikey=30301d38578cfcd4fe64ed9cc10024b6



ic| valueChangeCnt: 16
ic| logItemParam.fatalError: False
ic| logItemParam: LogItem(dateStr='2021-08-30', timeStampEnd=1630319850448.939, feedId=369278, length=18, fatalError=False, errorStr='', points=[], latestPoint=PointTT(time=0, temp=0))
ic| temperatureLog.py:223 in fillLogItem() at 20:37:31.634



Length is 17
PointTT(time=20.339, temp=15.11)
PointTT(time=20.356, temp=15.21)
PointTT(time=20.372, temp=15.22)
PointTT(time=20.389, temp=15.08)
PointTT(time=20.408, temp=15.09)
PointTT(time=20.425, temp=15.23)
PointTT(time=20.442, temp=15.11)
PointTT(time=20.458, temp=15.11)
PointTT(time=20.475, temp=15.1)
PointTT(time=20.494, temp=15.19)
PointTT(time=20.511, temp=15.23)
PointTT(time=20.528, temp=15.13)
PointTT(time=20.544, temp=15.21)
PointTT(time=20.564, temp=15.17)
PointTT(time=20.581, temp=15.12)
PointTT(time=20.597, temp=15.13)
PointTT(time=20.614, temp=15.21)
Reading side . . .
Url = http://emoncms.org/feed/data.json?id=369278&start=1630318770448.939&end=1630319850448.939&interval=5&skipmissing=1&limitinterval=1&apikey=30301d38578cfcd4fe64ed9cc10024b6



ic| valueChangeCnt: 16
ic| logItemParam.fatalError: False
ic| 'Setting lateness (day 1) instead of predictions'



Length is 17
PointTT(time=20.325, temp=14.92)
PointTT(time=20.342, temp=15.01)
PointTT(time=20.358, temp=14.94)
PointTT(time=20.375, temp=15.02)
PointTT(time=20.392, temp=15.03)
PointTT(time=20.408, temp=14.91)
PointTT(time=20.425, temp=14.91)
PointTT(time=20.442, temp=14.92)
PointTT(time=20.458, temp=15.07)
PointTT(time=20.475, temp=14.9)
PointTT(time=20.492, temp=14.91)
PointTT(time=20.508, temp=15.07)
PointTT(time=20.525, temp=15.02)
PointTT(time=20.542, temp=15.03)
PointTT(time=20.558, temp=14.93)
PointTT(time=20.575, temp=15.05)
PointTT(time=20.592, temp=15.02)

Temp at  20:35 is 15.1  (15 - 15)
Last  1 min: 0
Last  2 min: 0
Last  4 min: 0 ---
Last  8 min: 1
Want 125 in -6.1 hr (at 14:30)
Predicted finish time at  this rate: Tomorrow?
Predicted finish time at target rate: Tomorrow?
!! Running 640 minutes late !


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\azzle\documents\bigsoftware\python38-32\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\azzle\AppData\Local\Temp/ipykernel_28632/2883467170.py", line 53, in rememberConfig
    continuedBelow(virtualEvent)		# . . . for programmer's convenience (while keeping interpretter happy)
  File "C:\Users\azzle\AppData\Local\Temp/ipykernel_28632/2883467170.py", line 261, in continuedBelow
    getAndDisplayResults()
  File "C:\Users\azzle\AppData\Local\Temp/ipykernel_28632/2883467170.py", line 272, in getAndDisplayResults
    history = GetHistory(setupHasChanged, settingsShared)
  File "C:\Users\azzle\Documents\PatsThings\Python\Boiler rate\BoilerRate_linear9\gethistory.py", line 29, in __init__
    self.results = self.readEmon(self.setupHasChanged)
  File "C:\Users\azzle\Documents\PatsThings\Python\Boiler rate\BoilerRate_linear9\gethistory.py", line 124, in readEmon
    h